### Eventloop
Ein Eventloop ist eine Schleife, die z.B. all 100 Millisekunden prüft, ob
neue Events ( Tastendrücke, Mausklicks, Mausbewegungen, ...) stattgefunden haben und dann
entsprechend reagiert.

Wir benutzen einen Eventloop, um mir den Pfeiltasten eine Linie zu zeichnen.
Die Linie setzt sich jede Sekunde jeweils ein paar Pixel in
Richtung der zuletzt gedrücken Pfeiltaste fort.

Wir assozieren jede Pfeiltaste mit eine Richtungsvektor:
```python
key_direction = {'ArrowUp': (0, -5),
                 'ArrowDown': (0, 5),
                 'ArrowLeft': (-5, 0),
                 'ArrowRight': (5, 0),
                 }
```
Wird eine Pfeiltaste gedrückt, wird der entsprechende Richtungsvektor als erstes Element in eine Liste `event_queue` eingefügt. 
Ist der Eventqueue  nicht leer, entfernt 
der Eventloop den letzten Richtungsvektor, andernfalls wird der letzte Richtungsvektor beibehalten.

Der Richtungsvektor wird in der Variable `direction` gespeichert.
Dann wird der Vektor ausgebackt, `dx, dy = direction` und die Funktion
`move(dx, dy)` aufgerufen,
welche die Linie auf der Leinwand fortsetzt. 

Der Event-Loop wird mit Hilfe der Funktion `threading.Timer`.
Nachstehend definierte Funktion `event_loop` 
macht sich zu einem Thread, der mit etwas Verzögerung startet.
Und dieser Thread macht aus der Funktion erneut einen solchen Thread.
Dieser Vorgang wird gestoppt, wenn das `stop_event` gesetzt wird.


```python
def move(dx, dy):
    '''zeichne Linie von der aktuellen Position (x, y) 
       nach (x+dx, y+dy)
    '''
    
event_queue = [(0, -5), (5, 0)]


def event_loop(last_event=None, count=0):
    if event_queue:
        direction = event_queue.pop()
    else:
        direction = last_event

    dx, dy = direction
    move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(direction, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()            
```

***
**threading.Event**:  
Ein `threading.Event` ist im wesentlichen ein Flag, das entweder `True` oder `False` ist.
Es hat folgende Methoden:
- `stop_event = threading.Event()`: Erstellt ein `threading.Event`,
- `is_set()` : liefert `True` oder `False` (Default),
- `set()`    : `is_set()` liefert nun `True`,
- `clear()`  : `is_set()` liefert nun `False`.
***

In [ ]:
import threading


stop_event = threading.Event()
stop_event.is_set()

In [ ]:
stop_event.set()
stop_event.is_set()

In [ ]:
stop_event.clear()
stop_event.is_set()

In [1]:
from time import sleep
from ipywidgets import Output, Button
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)
button_stop = Button(description='Stop')
button_start = Button(description='Start')

state = {'pos': (50, 50), 'run_flag': True}
key_direction = {'ArrowUp': (0, -5),
                 'ArrowDown': (0, 5),
                 'ArrowLeft': (-5, 0),
                 'ArrowRight': (5, 0),
                 }

event_queue = []


def clear_run_flag():
    state['run_flag'] = False


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key in key_direction:
        direction = key_direction[key]
        event_queue.insert(0, direction)


def move(dx, dy):
    x0, y0 = state['pos']
    x, y = (x0+dx, y0+dy)

    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)

    if x < 0 or y < 0 or x > 100 or y > 100:
        canvas.fill_rect(0, 0, canvas.width, canvas.height)
        stop_event.set()


def event_loop():
    while state['run_flag']:
        if event_queue:
            direction = event_queue.pop()
            last_direction = direction
        else:
            direction = last_direction

        if direction is not None:
            dx, dy = direction
            move(dx, dy)

        sleep(1)

def start():
    right = (5, 0)
    event_queue.insert(0, right)
    canvas.clear()
    state['run_flag'] = True
    state['pos'] = (50, 50)
    # event_loop()


canvas.on_key_down(on_key_down)
button_stop.on_click(lambda bt: clear_run_flag())
# button_start.on_click(lambda bt: start())  # does not work!
display(canvas, button_stop, button_start, out)

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Button(description='Stop', style=ButtonStyle())

Button(description='Start', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [2]:
start()
event_loop()

In [ ]:
# gib eine Richtung vor und starte den Eventloop
right = (5, 0)
event_queue.insert(0, right)
canvas.clear()
state['run_flag'] = True
state['pos'] = (50, 50)
event_loop()

In [ ]:
state

### Aufgabe
Erstellen einen weitere Button `Start`. Beim Klicken auf diesen Button soll eine 
Funktion `start()` aufgerufen werden, welche
- die Leinwand löscht,
- `stop_event.clear()` aufruft,
- den Event-Queue gelöscht,
- `state['pos']` gleich `(50, 50)` setzt, 
- `(0, 5)` in Event-Queue einfügt,
- den Event-Loop startet und der Leinwand Fokus gibt.  

Leite Fehlermeldungen der Funktion `start()` ins Output-Widget `out` um.